In [5]:
!pip install tensorflow opencv-python matplotlib

DEPRECATION: Loading egg at /Users/dikpaal/anaconda3/lib/python3.11/site-packages/tf_pose-0.1.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path='3.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [93]:
# MAKE PREDICTIONS
def go():

    cap = cv2.VideoCapture("squat_11.mp4") # or cv2.VideoCapture(<name of file>) if not real time

    while cap.isOpened():
        ret, frame = cap.read()

        # reshape image

        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192) # this is the required size by the model
        input_image = tf.cast(img, dtype=tf.float32) # required by the model    

        # setup input and output

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # make predictions

        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    #     # draw edges

    #     draw_edges(frame, keypoints_with_scores, EDGES, 0.3)

    #     # draw keypoints

    #     draw_keypoints(frame, keypoints_with_scores, 0.3)

        # sqaut
        squat(frame, keypoints_with_scores)

        # TO EXTRACT COORDINATED, see the (MAKE POSE DETECTION)
        # for me, it is 1080, 1920 instead of 480, 640

        cv2.imshow('Camera Frame', frame)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
            
    for feedback in get_all_feedback():
        cv2.putText(frame, "Feedback: " + feedback, (7, 350), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3, cv2.LINE_AA)

    cap.release()
    cv2.destroyAllWindows()

In [78]:
# ## DRAW KEYPOINTS

# def draw_keypoints(frame, keypoints, confidence):
#     y, x, c = frame.shape
#     shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
#     for kp in shaped:
#         ky, kx, kp_confidence = kp
#         if (kp_confidence > confidence):
#             cv2.circle(frame, (int(kx), int(ky)), 3, (0, 255, 0), -1)

In [79]:
# DRAW EDGES

# {edge, color of edge}
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# def draw_edges(frame, keypoints, edges, conf_threshold):
#     y, x, c = frame.shape
#     shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
#     for edge, color in edges.items():
#         p1, p2 = edge
#         y1, x1, c1 = shaped[p1]
#         y2, x2, c2 = shaped[p2]
        
#         if (c1 > conf_threshold) & (c2 > conf_threshold):      
#             cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [80]:
# CALCULATE FORM

def get_coords(frame, keypoints):
    left_shoulder = np.array(keypoints[0][0][5][:2]*[400, 800]).astype(int)
    left_hip = np.array(keypoints[0][0][11][:2]*[400, 800]).astype(int)
    left_knee = np.array(keypoints[0][0][13][:2]*[400, 800]).astype(int)
    left_ankle = np.array(keypoints[0][0][15][:2]*[400, 800]).astype(int)
    
    right_shoulder = np.array(keypoints[0][0][6][:2]*[400, 800]).astype(int)
    right_hip = np.array(keypoints[0][0][12][:2]*[400, 800]).astype(int)
    right_knee = np.array(keypoints[0][0][14][:2]*[400, 800]).astype(int)
    right_ankle = np.array(keypoints[0][0][16][:2]*[400, 800]).astype(int)
    
    return left_shoulder, left_hip, left_knee, left_ankle, right_shoulder, right_hip, right_knee, right_ankle

def calculate_angle(c1, c2, c3):
    
    c1_c2 = np.sqrt((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2)
    c1_c3 = np.sqrt((c1[0] - c3[0]) ** 2 + (c1[1] - c3[1]) ** 2)
    c2_c3 = np.sqrt((c2[0] - c3[0]) ** 2 + (c2[1] - c3[1]) ** 2)
    
    num = c1_c2 ** 2 + c2_c3 ** 2 - c1_c3 ** 2
    den = 2 * c1_c2 * c2_c3
    
    angle = np.arccos(num / den)
    
    angle *= 180 / np.pi
    
    return angle

In [103]:
# EXERCISE

def squat(frame, keypoints):
    left_shoulder, left_hip, left_knee, left_ankle, right_shoulder, right_hip, right_knee, right_ankle = get_coords(frame, keypoints)

    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
#     cv2.putText(frame, "Left hip angle: " + str(left_hip_angle), (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3, cv2.LINE_AA)
#     cv2.putText(frame, "Right hip angle: " + str(right_hip_angle), (7, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3, cv2.LINE_AA)
    
#     cv2.putText(frame, "Left knee angle: " + str(left_knee_angle), (7, 210), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3, cv2.LINE_AA)
#     cv2.putText(frame, "Right knee angle: " + str(right_knee_angle), (7, 280), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3, cv2.LINE_AA)
    
    squat_form(frame, left_hip_angle, right_hip_angle, left_knee_angle, right_knee_angle)


In [106]:
# Squat Form

all_feedback = []

def squat_form(frame, LHA, RHA, LKA, RKA):
    feedback = ''
    
    if (LHA <= 90 and RHA <= 90) and (abs(LKA - RKA) <= 10) and (abs(LHA - RHA) <= 10):
        feedback = "Good form!"
    
    elif (160 <= LKA <= 180 and 160 <= RKA <= 180):
        feedback = "Good form!"
    
    # Check for hips not low enough
    elif LHA > 90 or RHA > 90:
        feedback = "Squat lower"
        all_feedback.append(feedback)
    
    # Check for knees caving in (symmetry issues as a proxy)
    elif abs(LKA - RKA) > 10:
        feedback = "Keep your knees out."
        all_feedback.append(feedback)
    
    # Check for symmetry issues
    elif abs(LHA - RHA) > 10:
        feedback = "Ensure both sides move symmetrically"
        all_feedback.append(feedback)
        
#     cv2.putText(frame, "Feedback: " + feedback, (7, 280), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3, cv2.LINE_AA)

    
    size_of_all_feedback = len(all_feedback)
    if (size_of_all_feedback > 5 and all_feedback[size_of_all_feedback - 1] == all_feedback[size_of_all_feedback - 2] == all_feedback[size_of_all_feedback - 3] == all_feedback[size_of_all_feedback - 4] == all_feedback[size_of_all_feedback - 5]):
        
        cleaned_all_feedback = clean_all_feedback(all_feedback)
        display_all_feedback(frame, cleaned_all_feedback)

def get_all_feedback():
    return all_feedback

def clean_all_feedback(feedbacks):
    temp = []
    
    for feedback in feedbacks:
        if (feedback not in temp):
            temp.append(feedback)
    return temp

def display_all_feedback(frame, feedbacks):
    y = 70
    for line in feedbacks:
        cv2.putText(frame, "> " + line, (7, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3, cv2.LINE_AA)
        y += 70

In [107]:
go()

AttributeError: 'NoneType' object has no attribute 'copy'